In [134]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
from types import FunctionType
from math import erf

%matplotlib inline

In [140]:
class Perceptron:
    def __init__(self, n_inputs, learning_rate=0.1):
        self.weights = np.zeros(n_inputs)
        self.bias = 0
        self.learning_rate = learning_rate
    
    def forward(self, inputs):
        return np.heaviside(self.weights @ inputs + self.bias, 0)
    
    def backward(self, inputs, error):
        self.weights += self.learning_rate * error * inputs
        self.bias += self.learning_rate * error


In [144]:
#PERCEPTRON

inputs = np.array([[2, 3], [-1, 2]])
outputs = np.array([1, 0])

neuron = Perceptron(inputs.shape[1])

#train
for input, output in zip(inputs, outputs):
    pred = neuron.forward(input)
    error = output - pred    
    neuron.backward(input, error)

#test 
for input, output in zip(inputs, outputs):
    pred = neuron.forward(input)
    print(f"{input} -> {output:.2f}\nPredicted -> {pred}")

[2 3] -> 1.00
Predicted -> 1.0
[-1  2] -> 0.00
Predicted -> 0.0


In [119]:
class Neuron:
    def __init__(self, n_inputs: np.ndarray, 
                 learning_rate: float=0.1):
        self.weights = np.random.uniform(-0.1, 0.1, size=n_inputs)
        self.bias = np.random.uniform(-0.1, 0.1)
        self.learning_rate = learning_rate
    
    def __call__(self, inputs: np.ndarray) -> np.ndarray:
        return self.weights @ inputs + self.bias
    
    def backward(self, inputs: np.ndarray, 
                 error: np.ndarray) -> None:
        dC_dw = inputs.T * error
        dC_db = np.sum(error)
        self.weights -= self.learning_rate * dC_dw
        self.bias -= self.learning_rate * dC_db

In [152]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return (2 / (1 + np.exp(-2 * x))) - 1

def gelu(x):
    return x * 0.5 * (1.0 + erf(x / 1.41421))

def swish(x):
    return x * sigmoid(x)

def leakyrelu(x, negative_slope=0.01):
    return np.where(x >= 0, x, negative_slope * x)

In [153]:
def train(inputs: np.ndarray, outputs: np.ndarray, 
          epochs: int=100, lr: float=0.1, 
          activation: FunctionType=relu) -> None:
    
    model = Neuron(inputs.shape[1], learning_rate=lr)

    for _ in tqdm.tqdm(range(epochs)):
        total_loss = 0
        for input, output in zip(inputs, outputs):
            pred = activation(model(input))
            error = pred - output
            model.backward(input, error)
            total_loss += (model(input) - output) ** 2

    for input, output in zip(inputs, outputs):
        pred = activation(model(input))
        print(f"{input} -> {output:.2f}\nPredicted -> {pred:.2f}")

In [154]:
EPOCH = 1000
LEARNING_RATE = 0.1

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 0, 0, 1])

train(X, y, epochs=EPOCH, lr=LEARNING_RATE, activation=relu)

  0%|          | 0/1000 [00:00<?, ?it/s]

[0 0] -> 0.00
Predicted -> 0.00
[0 1] -> 0.00
Predicted -> 0.00
[1 0] -> 0.00
Predicted -> 0.00
[1 1] -> 1.00
Predicted -> 1.00
